<a href="https://colab.research.google.com/github/Blaizzy/BiSeNet-Implementation/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras 
import tensorflow as tf
from keras.applications.xception import Xception
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Conv2D, Input, Dense, Dropout, multiply, Dot, Concatenate,Add, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, AveragePooling2D, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from tensorflow.python.keras.callbacks import TensorBoard

from keras import backend as K
K.tensorflow_backend._get_available_gpus()


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
#Accessing my google drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
          
# Open a file



def display_one(a, title1 = "Original"):
    plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.show()



# **Labels**

In [0]:
import csv
import numpy as np

#Change the dir to the file dir
os.chdir('/content/drive/My Drive/BiSeNet')

# Read labels from csv and load into lists
labels = []
with open("/content/drive/My Drive/BiSeNet/classes.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',') # change contents to floats
    for row in reader: # each row is a list
        labels.append(row)
        
del labels[0]

labels = np.array(labels)        
print("No of labels",len(labels), "\n----------------------------------------")
num_classes = 32
labels_string = labels[:,0]
labels_string.tolist()
str = ' '.join(labels_string)
print(str[:31],"....\n-----------------------------------------")

# One_hot encode labels
y = keras.preprocessing.text.one_hot(str, num_classes, 
                                     filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~  ',
                                     split=',')    
print("one-hot",y,"\n----------------------------------")

    
y = keras.utils.to_categorical(y, num_classes=num_classes)
print("categorical")
for i in y[:5]:
    print(i)
    print("checking categorical to one-hot", np.argmax(i))
    


No of labels 32 
----------------------------------------
Animal Archway Bicyclist Bridge ....
-----------------------------------------
one-hot [29, 4, 22, 23, 22, 31, 24, 18, 25, 29, 18, 17, 21, 7, 10, 26, 7, 22, 21, 13, 19, 31, 10, 26, 17, 17, 17, 23, 23, 11, 31, 28, 1, 30, 24] 
----------------------------------
categorical
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0.]
checking categorical to one-hot 29
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
checking categorical to one-hot 4
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
checking categorical to one-hot 22
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0.]
checking categorical to one-hot 23
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
checking categorical to one-hot 22


# **Image** **Preprocessing** 
import Preprocessing python file

In [0]:
#Change dir to the file dir
os.chdir('/content/drive/My Drive/Colab Notebooks/python files/')
import images


# load image paths into list
# return train[0], train_mask[1], val[2], val_mask[3] 
dataset = images.load('/content/drive/My Drive/BiSeNet/dataset')
sample = dataset[0] # Train dir
print("First three images in our Train dir")
for i in sample[:3]:
    print("checking files....", i)
    

a,b,c,d = dataset[0], dataset[1], dataset[2], dataset[3]


total = len(a+b+c+d) 
print("\n Number of Images", total) 

load() 4 s
First three images in our Train dir
checking files.... /content/drive/My Drive/BiSeNet/dataset/train/0001TP_006690.png
checking files.... /content/drive/My Drive/BiSeNet/dataset/train/0001TP_006720.png
checking files.... /content/drive/My Drive/BiSeNet/dataset/train/0001TP_006750.png

 Number of Images 973


In [0]:
with tf.device('/GPU:0'):
  # Preprocess 
  # Read images from path and resize them
  # This step takes a while so hang in there
  train = images.preprocess(a)
  train_mask = images.preprocess(b)
  val = images.preprocess(c)
  val_mask = images.preprocess(d)




Before resize (360, 480, 3)

---------------------------------------------
 Resized (299, 299, 3) 
 ---------------------------------------------

preprocess() 253 s

Before resize (360, 480, 3)

---------------------------------------------
 Resized (299, 299, 3) 
 ---------------------------------------------

preprocess() 113 s


In [0]:
# Transform images 3D into 4D np.array
# firt Dim is the number of pictures
# Eases the batching process
train = np.array(train)
train_mask = np.array(train_mask)
val = np.array(val)
val_mask = np.array(val_mask)

print('Train', train.shape,'\nTrain_mask', train_mask.shape,
      '\nVal', val.shape,'\nVal_mask',val_mask.shape)

Train (394, 299, 299, 3) 
Train_mask (367, 299, 299, 3) 
Val (101, 299, 299, 3) 
Val_mask (111, 299, 299, 3)


# **Augmenting the Dataset**
Our dataset consists of 973 images

In [0]:

# we create four instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)

train_datagen = ImageDataGenerator(**data_gen_args)
train_mask_datagen = ImageDataGenerator(**data_gen_args)

val_datagen = ImageDataGenerator(**data_gen_args)
val_mask_datagen = ImageDataGenerator(**data_gen_args)


batch_size = 10

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
train_datagen.fit(train, augment=True, seed=seed)
train_mask_datagen.fit(train_mask, augment=True, seed=seed)
val_datagen.fit(val, augment=True, seed=seed)
val_mask_datagen.fit(val_mask, augment=True, seed=seed)

# Zip images (x) and their masks (y)
train_generator = zip(train, train_mask)
val_generator = zip(val, val_mask)


# **Model**


In [0]:
# ARM (Attention Refinement Module) 
# Refines features at each stage of the Context path

def ARM(inputs, n_filters):
  
  arm = AveragePooling2D(pool_size=(1, 1), padding='same', data_format= 'channels_last')(inputs)
  arm = Conv2D(n_filters, (1,1), strides= 1 )(arm)
  arm = BatchNormalization()(arm)
  arm = Activation('sigmoid')(arm)
  arm = multiply([inputs,arm])
  
  return arm

In [0]:
# FFM (Feature Fusion Module)
# Fuses the low level features of Spatial Path and high level 
# features of Context Path

def FFM(input_sp, input_cp, n_classes):
  
    ffm = Concatenate(axis= -1)([input_sp,input_cp])
    conv = Conv2D(n_classes, (3,3), strides = 2)(ffm)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    avg = AveragePooling2D(pool_size=(1, 1), padding = 'same', data_format='channels_last')(conv)
    conv_1 = Conv2D(n_classes, (1,1), strides = 1)(avg)
    conv_1 = Activation('relu')(conv_1)
    conv_1 = Conv2D(n_classes, (1,1), strides = 1)(conv_1)
    conv_1 = Activation('sigmoid')(conv_1)
    ffm = multiply([conv,conv_1])
    ffm = Add()([conv, ffm])
    
    return ffm

In [0]:
# Spatial path to extract rich spatial information
# Composed of 3 conv of stride 2
# Conv + batch + ReLU
inputs = Input(shape=(224, 224, 3))

conv1 = Conv2D(32,(2,2), strides=2, data_format='channels_last', name= 'spatial_conv1')(inputs)
conv1 = BatchNormalization(name= 'spatial_bn1')(conv1)
conv1 = Activation('relu', name= 'spatial_act1')(conv1)

conv2 = Conv2D(64,(2,2),strides=2, name= 'spatial_conv2')(conv1)
conv2 = BatchNormalization(name= 'spatial_bn2')(conv2)
conv2 = Activation('relu', name= 'spatial_act2')(conv2)

conv3 = Conv2D(256,(2,2), strides=2, name= 'spatial_conv3')(conv2)
conv3 = BatchNormalization(name= 'spatial_bn3')(conv3)
conv3 = Activation('relu')(conv3)
#------------------------------------

# Context path to extract enough receptive field
# Composed of pre-trained Xception39 model as the backbone
# Global avg pooling at the tail of the Xception
base_model = Xception(weights='imagenet',input_shape= (224,224,3), include_top=False)

for layer in  base_model.layers:
    layer.trainable = False

# Attention Refinement Module
layer_13 = base_model.get_layer('block13_pool').output

layer_14 = base_model.output

# Combine the up-sampled output feature of Global avg pooling and Xception features
tail_avg = GlobalAveragePooling2D()(layer_14)
tail_upS = UpSampling2D(size=(2, 2), data_format='channels_last', interpolation='nearest')(layer_14) 
tail = Add()([tail_avg, tail_upS])

layer_13 = ARM(layer_13, 1024)
layer_14 = ARM(layer_14, 2048)

layer_13 = UpSampling2D(size= 2 , data_format='channels_last', interpolation='nearest')(layer_13)
layer_14 = UpSampling2D(size= 2 , data_format='channels_last', interpolation='nearest')(layer_14)

context_features  = Concatenate(axis = -1)([layer_14,layer_13])
context_features = Concatenate(axis = -1)([context_features,tail])

context_features = UpSampling2D(size= 2 , data_format='channels_last', interpolation='nearest')(context_features)

# Feature Fusion Module
FFM = FFM(conv3, context_features, 32)

# Upscaling To intial image size
output = UpSampling2D(size=(16,16) , data_format='channels_last', interpolation='nearest')(FFM)
print(output)
#-----------------------------------------------

# Model Summary
bisenet = Model(input = [inputs, base_model.input], output = output )
print(bisenet.summary())



Tensor("activation_10/Relu:0", shape=(?, 28, 28, 256), dtype=float32)
Tensor("up_sampling2d_13/ResizeNearestNeighbor:0", shape=(?, 28, 28, 5120), dtype=float32)
Tensor("up_sampling2d_14/ResizeNearestNeighbor:0", shape=(?, 208, 208, 32), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
_______________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("up...)`


In [0]:

# Custom loss
def softmax_loss(y_true, y_pred):
  
  
 


# Custom optimizer
sgd = SGD(lr=0.001, decay=1e-4, momentum=0.9, nesterov=False)




In [0]:
bisenet.compile(optimizer = sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
#Train and visualizing loss and acc in Tensorboard
tensorboard = TensorBoard(log_dir ="logs/{}".format(time()))

with tf.device('/GPU:0'):
  history = bisenet.fit_generator(
          train_generator,
          steps_per_epoch=100 // batch_size,
          epochs=50,
          validation_data=val_generator,
          validation_steps=100 // batch_size,
          callbacks = [tensorboard])
  bisenet.save_weights('first_try.h5')


  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

Epoch 1/50


ValueError: ignored